In [ ]:
import tensorflow as tf
import os
import numpy as np
n_input=30
n_hidden_1=30
n_hidden_2=30
n_classes=3
real_input=tf.placeholder(tf.float32,[569,30],"real_input")
real_label=tf.placeholder(tf.int32,[569],"real_label")
real_label=tf.one_hot(real_label,3)
fake_input=tf.get_variable("fake_input",[569,30],dtype=tf.float32,initializer=tf.random_normal_initializer(0, 0.02))
fake_label=tf.constant(2,tf.int32,[569],"fake")
fake_label=tf.one_hot(fake_label,3)
#这里应该是
def multilayer_perceptron(x,name):
    weigh1=tf.get_variable("weight1",[n_input,n_hidden_1],dtype=tf.float32,initializer=tf.random_normal_initializer(0, 0.02))
    # Hidden layer with RELU activation
    bias1=tf.get_variable("bias1",[n_hidden_1],dtype=tf.float32,initializer=tf.random_normal_initializer(0,0.02))
    
    layer_1 = tf.add(tf.matmul(x, weigh1), bias1)
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    weigh2=tf.get_variable("weight2",[n_hidden_1,n_hidden_2],dtype=tf.float32,initializer=tf.random_normal_initializer(0, 0.02))
    bias2=tf.get_variable("bias2",[n_hidden_2],dtype=tf.float32,initializer=tf.random_normal_initializer(0,0.02))
    layer_2 = tf.add(tf.matmul(layer_1, weigh2), bias2)
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    if name=="generator":
        out_layer=layer_2
    else:
        weigh3=tf.get_variable("weight2",[n_hidden_2,n_classes],dtype=tf.float32,initializer=tf.random_normal_initializer(0, 0.02))
        bias3=tf.get_variable("bias3",[n_classes],dtype=tf.float32,initializer=tf.random_normal_initializer(0,0.02))
        out_layer = tf.matmul(layer_2, weigh3) + bias3
    return out_layer

with tf.name_scope("generator"):
    out=multilayer_perceptron(batch_input,"generator")

def discriminator(batch_input,batch_label):
    with tf.name_scope("discriminator"):
        out=multilayer_perceptron(batch_input,"discriminator")
        cross_entropy=tf.nn.softmax_cross_entropy_with_logits(logits=out,labels=batch_label)
        return cross_entropy
with tf.name_scope("real_discriminator"):
    with tf.variable_scope("discriminator"):
        real_predict=discriminator(real_input,real_label)
with tf.name_scope("fake_discriminator"):
    with tf.variable_scope("discriminator",reuse=True):
        fake_predict=discriminator(fake_input,fake_label)
with tf.name_scope("genertor_loss"):
    gen_loss=tf.reduce_mean(-tf.log(fake_predict))

with tf.name_scope("discriminator_loss"):
    discrim_loss=tf.reduce_mean(-(tf.log(real_predict)+tf.log(1-fake_label)))

with tf.name_scope("generator_train"):
    vars_gen=[var for var in tf.trainable_variables() if var.name.startswith("generator")]
    gen_optim=tf.train.AdamOptimizer(0.001,0.9)
    gradients=gen_optim.compute_gradients(gen_loss,var_list=vars_gen)
    gen_train=gen_optim.apply_gradients(vars_gen)
with tf.name_scope("discriminator_train"):
    discrim_vars=[var for var in tf.trainable_variables() if var.name.startswith("discriminator")]
    discrim_optim=tf.train.AdamOptimizer(0.001,0.9)
    gradients=discrim_optim.compute_gradients(discrim_loss,var_list=discrim_vars)
    discrim_train=discrim_optim.apply_gradients(discrim_vars)

In [9]:
import numpy as np
data=np.genfromtxt("/home/zzq/research/gan_learning/brancer.csv",dtype='unicode',delimiter=',')
print data.shape

(569, 32)


In [10]:
pre_data=data[:,2:]
label=data[:,1]
print label

[u'M' u'M' u'M' u'M' u'M' u'M' u'M' u'M' u'M' u'M' u'M' u'M' u'M' u'M' u'M'
 u'M' u'M' u'M' u'M' u'B' u'B' u'B' u'M' u'M' u'M' u'M' u'M' u'M' u'M' u'M'
 u'M' u'M' u'M' u'M' u'M' u'M' u'M' u'B' u'M' u'M' u'M' u'M' u'M' u'M' u'M'
 u'M' u'B' u'M' u'B' u'B' u'B' u'B' u'B' u'M' u'M' u'B' u'M' u'M' u'B' u'B'
 u'B' u'B' u'M' u'B' u'M' u'M' u'B' u'B' u'B' u'B' u'M' u'B' u'M' u'M' u'B'
 u'M' u'B' u'M' u'M' u'B' u'B' u'B' u'M' u'M' u'B' u'M' u'M' u'M' u'B' u'B'
 u'B' u'M' u'B' u'B' u'M' u'M' u'B' u'B' u'B' u'M' u'M' u'B' u'B' u'B' u'B'
 u'M' u'B' u'B' u'M' u'B' u'B' u'B' u'B' u'B' u'B' u'B' u'B' u'M' u'M' u'M'
 u'B' u'M' u'M' u'B' u'B' u'B' u'M' u'M' u'B' u'M' u'B' u'M' u'M' u'B' u'M'
 u'M' u'B' u'B' u'M' u'B' u'B' u'M' u'B' u'B' u'B' u'B' u'M' u'B' u'B' u'B'
 u'B' u'B' u'B' u'B' u'B' u'B' u'M' u'B' u'B' u'B' u'B' u'M' u'M' u'B' u'M'
 u'B' u'B' u'M' u'M' u'B' u'B' u'M' u'M' u'B' u'B' u'B' u'B' u'M' u'B' u'B'
 u'M' u'M' u'M' u'B' u'M' u'B' u'M' u'B' u'B' u'B' u'M' u'B' u'B' u'M' u'M'
 u'B' u'M' u

In [11]:
for i in range(len(label)):
    if label[i]=='M':
        label[i]=1.
    else:
        label[i]=0.

In [12]:
print label

[u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0'
 u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'0.0'
 u'0.0' u'0.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0'
 u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'0.0' u'1.0' u'1.0'
 u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'1.0' u'0.0' u'1.0' u'0.0' u'0.0'
 u'0.0' u'0.0' u'0.0' u'1.0' u'1.0' u'0.0' u'1.0' u'1.0' u'0.0' u'0.0'
 u'0.0' u'0.0' u'1.0' u'0.0' u'1.0' u'1.0' u'0.0' u'0.0' u'0.0' u'0.0'
 u'1.0' u'0.0' u'1.0' u'1.0' u'0.0' u'1.0' u'0.0' u'1.0' u'1.0' u'0.0'
 u'0.0' u'0.0' u'1.0' u'1.0' u'0.0' u'1.0' u'1.0' u'1.0' u'0.0' u'0.0'
 u'0.0' u'1.0' u'0.0' u'0.0' u'1.0' u'1.0' u'0.0' u'0.0' u'0.0' u'1.0'
 u'1.0' u'0.0' u'0.0' u'0.0' u'0.0' u'1.0' u'0.0' u'0.0' u'1.0' u'0.0'
 u'0.0' u'0.0' u'0.0' u'0.0' u'0.0' u'0.0' u'0.0' u'1.0' u'1.0' u'1.0'
 u'0.0' u'1.0' u'1.0' u'0.0' u'0.0' u'0.0' u'1.0' u'1.0' u'0.0' u'1.0'
 u'0.0' u'1.0' u'1.0' u'0.0' u'1.0' u'1.0' u'0.0' u'0.0' u'1.0' u'0.0'
 u'0.0

In [13]:
print pre_data.shape

(569, 30)


In [2]:
import numpy as np
import random
print random.randint(0,1)

0


In [3]:
import tensorflow as tf
data = tf.constant( [ [1,2,3], [4,5,6] ] )
zip(data,2)

TypeError: zip argument #1 must support iteration